In [485]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np

In [486]:
RANDOM_STATE = 42

In [487]:
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)
simplefilter("ignore", category=FutureWarning)

In [488]:
dataset = load_boston()
X = pd.DataFrame(dataset.data)
X.columns = dataset.feature_names
y = dataset.target

1. Разделите выборку на обучающую и тестовую в отношении 80%/20%

In [489]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=RANDOM_STATE)

2. Обучите стандартную регрессию, а также Ridge и  Lasso и параметрами по умолчанию и выведите их R2 на тестовой выборке

In [490]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, RidgeCV, LassoCV
from sklearn.metrics import r2_score

In [491]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_linear_reg = lr.predict(X_test)
print('linear regression r2 =', r2_score(y_test, y_linear_reg))

linear regression r2 = 0.7112260057484912


In [492]:
ridge = Ridge()
ridge.fit(X_train, y_train)
y_ridge = ridge.predict(X_test)
print('ridge r2 =', r2_score(y_test, y_ridge))

ridge r2 = 0.7041586727559435


In [493]:
lasso = Lasso()
lasso.fit(X_train, y_train)
y_lasso = lasso.predict(X_test)
print('lasso r2 =', r2_score(y_test, y_lasso))

lasso r2 = 0.655906082915434


3. Для Ridge и Lasso подберите коэффициент регуляризации(используйте GridSearchCV, RidgeCV, LassoCV) в пределах от $10^{-5}$ до $10^5$ (по степеням 10). Посчитайте R2 на тестовой выборке по лучшим моделям и сравните с предыдущими результатами. Напишите как изменился результат

In [494]:
alphas = [10**a for a in range(-5, 6, 1)]
ridge_cv = RidgeCV(alphas=alphas)
ridge_cv.fit(X_train, y_train)
y_ridge_cv = ridge_cv.predict(X_test)
print('Best alpha:', ridge_cv.alpha_)
print('RidgeCV r2 =', r2_score(y_test, y_ridge_cv))

parameters = {'alpha': alphas}
grid_ridge = GridSearchCV(ridge, parameters)
grid_ridge.fit(X_train, y_train)
y_grid_ridge = grid_ridge.predict(X_test)
print('\nBest alpha:', grid_ridge.best_params_['alpha'])
print('GridSearchCV ridge r2 =', r2_score(y_test, y_grid_ridge))

Best alpha: 0.01
RidgeCV r2 = 0.7111350610032618

Best alpha: 1e-05
GridSearchCV ridge r2 = 0.7112259149897334


In [495]:
lasso_cv = LassoCV(alphas=alphas)
lasso_cv.fit(X_train, y_train)
y_lasso_cv = lasso_cv.predict(X_test)
print('Best alpha:', lasso_cv.alpha_)
print('LassoCV r2 =', r2_score(y_test, y_lasso_cv))

parameters = {'alpha': alphas}
grid_lasso = GridSearchCV(lasso, parameters)
grid_lasso.fit(X_train, y_train)
y_grid_lasso = grid_lasso.predict(X_test)
print('Best alpha:', grid_lasso.best_params_['alpha'])
print('GridSearchCV lasso r2 =', r2_score(y_test, y_grid_lasso))

Best alpha: 1e-05
LassoCV r2 = 0.7112241989771797
Best alpha: 1e-05
GridSearchCV lasso r2 = 0.7112241989771797


In [496]:
# Для ridge немного лучше отработал GridSearchCV и улучшил результат стандартной модели Lasso на 6%.

# Для lasso, и GridSearchCV, и LassoCV выбрали одинаковую альфу и дали
# тот же результат, который оказался лучше стандартного значения альфы Lasso на 5%.

4. Проведите масштабирование выборки(используйте Pipeline, StandardScaler, MinMaxScaler), посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [497]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler
pipe_sc = Pipeline([('scaler', StandardScaler())])
pipe_min_max = Pipeline([('scaler', MinMaxScaler())])

In [498]:
X_train_sc = pipe_sc.fit_transform(X_train)
X_test_sc = pipe_sc.fit_transform(X_test)
X_train_min_max = pipe_min_max.fit_transform(X_train)
X_test_min_max = pipe_min_max.fit_transform(X_test)

In [499]:
# ~~~~~~ StandardScaler ~~~~~~~
lr = LinearRegression()
lr.fit(X_train_sc, y_train)
y_linear_reg = lr.predict(X_test_sc)
print('linear regression r2 =', r2_score(y_test, y_linear_reg))

ridge = Ridge()
ridge.fit(X_train_sc, y_train)
y_ridge = ridge.predict(X_test_sc)
print('ridge r2 =', r2_score(y_test, y_ridge))

lasso = Lasso()
lasso.fit(X_train_sc, y_train)
y_lasso = lasso.predict(X_test_sc)
print('lasso r2 =', r2_score(y_test, y_lasso))

linear regression r2 = 0.6761000049033605
ridge r2 = 0.6757707272848222
lasso r2 = 0.6189573403855027


In [500]:
# ~~~~~~ MinMaxScaler ~~~~~~~
lr = LinearRegression()
lr.fit(X_train_min_max, y_train)
y_linear_reg = lr.predict(X_test_min_max)
print('linear regression r2 =', r2_score(y_test, y_linear_reg))

ridge = Ridge()
ridge.fit(X_train_min_max, y_train)
y_ridge = ridge.predict(X_test_min_max)
print('ridge r2 =', r2_score(y_test, y_ridge))

lasso = Lasso()
lasso.fit(X_train_min_max, y_train)
y_lasso = lasso.predict(X_test_min_max)
print('lasso r2 =', r2_score(y_test, y_lasso))

linear regression r2 = 0.6975456525264927
ridge r2 = 0.7016069171646895
lasso r2 = 0.2535763354600139


In [501]:
# По результатам видно, что масшабирование выборки дало нам ухудшение работы моделей.

5. Подберите коэффициент регуляризации для Ridge и Lasso на масштабированных данных, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [502]:
# ~~~~~~ StandardScaler ~~~~~~
lasso_cv = LassoCV(alphas=alphas)
lasso_cv.fit(X_train_sc, y_train)
y_lasso_cv = lasso_cv.predict(X_test_sc)
print('Best alpha:', lasso_cv.alpha_)
print('LassoCV r2 =', r2_score(y_test, y_lasso_cv))

ridge_cv = RidgeCV(alphas=alphas)
ridge_cv.fit(X_train_sc, y_train)
y_ridge_cv = ridge_cv.predict(X_test_sc)
print('Best alpha:', ridge_cv.alpha_)
print('RidgeCV r2 =', r2_score(y_test, y_ridge_cv))

Best alpha: 1e-05
LassoCV r2 = 0.676099586305966
Best alpha: 10.0
RidgeCV r2 = 0.6730214300158306


In [503]:
# ---------- MinMaxScaler
lasso_cv = LassoCV(alphas=alphas)
lasso_cv.fit(X_train_min_max, y_train)
y_lasso_cv = lasso_cv.predict(X_test_min_max)
print('Best alpha:', lasso_cv.alpha_)
print('LassoCV r2 =', r2_score(y_test, y_lasso_cv))

ridge_cv = RidgeCV(alphas=alphas)
ridge_cv.fit(X_train_min_max, y_train)
y_ridge_cv = ridge_cv.predict(X_test_min_max)
print('Best alpha:', ridge_cv.alpha_)
print('RidgeCV r2 =', r2_score(y_test, y_ridge_cv))

Best alpha: 1e-05
LassoCV r2 = 0.6975492304020645
Best alpha: 0.1
RidgeCV r2 = 0.6990278205880137


In [504]:
# По результатам можно увидеть, что масштабиррование ухудшило прогноз, но можно заметить, что MinMaxScaler работает лучше, чем 
# StandartScaler примерно на 2%

6. Добавьте попарные произведения признаков и их квадраты (используйте PolynomialFeatures) на масштабированных признаках, посчитайте R2 и сравните с предыдущими результатами. Напишите как изменился результат

In [505]:
from sklearn.preprocessing import PolynomialFeatures
pipe_sc = Pipeline([('scaler', StandardScaler()), ('preprocessing', PolynomialFeatures())])
pipe_min_max = Pipeline([('scaler', MinMaxScaler()), ('preprocessing', PolynomialFeatures())])

In [506]:
X_train_sc = pipe_sc.fit_transform(X_train)
X_test_sc = pipe_sc.fit_transform(X_test)
X_train_min_max = pipe_min_max.fit_transform(X_train)
X_test_min_max = pipe_min_max.fit_transform(X_test)

In [507]:
# ~~~~~~ StandardScaler ~~~~~~~
lr = LinearRegression()
lr.fit(X_train_sc, y_train)
y_linear_reg = lr.predict(X_test_sc)
print('linear regression r2 =', r2_score(y_test, y_linear_reg))

ridge = Ridge()
ridge.fit(X_train_sc, y_train)
y_ridge = ridge.predict(X_test_sc)
print('ridge r2 =', r2_score(y_test, y_ridge))

lasso = Lasso()
lasso.fit(X_train_sc, y_train)
y_lasso = lasso.predict(X_test_sc)
print('lasso r2 =', r2_score(y_test, y_lasso))

linear regression r2 = -1.3439177343227555e+24
ridge r2 = 0.7229070506730969
lasso r2 = 0.7448728358554388


In [508]:
# ~~~~~~ MinMaxScaler ~~~~~~~
lr = LinearRegression()
lr.fit(X_train_min_max, y_train)
y_linear_reg = lr.predict(X_test_min_max)
print('linear regression r2 =', r2_score(y_test, y_linear_reg))

ridge = Ridge()
ridge.fit(X_train_min_max, y_train)
y_ridge = ridge.predict(X_test_min_max)
print('ridge r2 =', r2_score(y_test, y_ridge))

lasso = Lasso()
lasso.fit(X_train_min_max, y_train)
y_lasso = lasso.predict(X_test_min_max)
print('lasso r2 =', r2_score(y_test, y_lasso))

linear regression r2 = 0.5675609165683645
ridge r2 = 0.8189284783995134
lasso r2 = 0.2578825732907113


In [509]:
# Изучив результаты попарного произведения признаков, можно сделать вывод о том, что для обычной линейной регрессии мы получили
# существенное ухудшение результатов, однако для ridge мы получили наиболее лучший результат, чем за все предыдущие шаги, 
# для lasso наилучшем оказалось использование стандартного масштабирования.

7. Подберите наилучшую модель (используйте Pipeline, GridSearchSCV) подбирая тип регуляризации (L1,L2), коэффициент регуляризации, метод масштабирования и степень полинома в PolynomialFeatures. Выведите итоговые параметры и результат R2. Напишите как изменился R2 по сравнению с предыдущими экспериментами

In [513]:
def create_pipe(model_cls, scaler_cls, model_name):
    pipe_sc = Pipeline([
        ('scaler', scaler_cls()), 
        ('preprocessing', PolynomialFeatures()), 
        ('model', model_cls())
    ])
    pipe_sc.score = r2_score
    param_grid = {'model__alpha': alphas, 
                  'preprocessing__degree': [1, 2, 3, 4]}
    grid_search = GridSearchCV(pipe_sc, param_grid=param_grid)
    grid_search.fit(X_train, y_train)

    y_grid = grid_search.predict(X_test)
    print(f'Best params for {model_name}: ', grid_search.best_params_)
    print(f'{model_name} r2:', r2_score(y_test, y_grid))

In [514]:
create_pipe(Lasso, MinMaxScaler, 'Lasso/MinMaxScaler')
create_pipe(Lasso, StandardScaler, 'Lasso/StandardScaler')
create_pipe(Ridge, MinMaxScaler, 'Lasso/MinMaxScaler')
create_pipe(Ridge, StandardScaler, 'Lasso/StandardScaler')

Best params for Lasso/MinMaxScaler:  {'model__alpha': 0.001, 'preprocessing__degree': 3}
Lasso/MinMaxScaler r2: 0.8439380295976925
Best params for Lasso/StandardScaler:  {'model__alpha': 0.1, 'preprocessing__degree': 2}
Lasso/StandardScaler r2: 0.8055776105496003
Best params for Lasso/MinMaxScaler:  {'model__alpha': 0.1, 'preprocessing__degree': 4}
Lasso/MinMaxScaler r2: 0.8352016738695145
Best params for Lasso/StandardScaler:  {'model__alpha': 10, 'preprocessing__degree': 2}
Lasso/StandardScaler r2: 0.8038169683868304


In [199]:
# Таким образом, лучшая модель - Lasso c MinMaxScaler. 
# По сравнению с исходным экспериментом (стандартной моделью Lasso) прогноз улучшился примерно на 20%.

http://archive.ics.uci.edu/ml/datasets/Adult

In [515]:
link = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/adult-all.csv'
data = pd.read_csv(link, header=None)

In [516]:
data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


8. Разделите выборку на признаки и целевую переменную(колонка со зачениями {<=50K,>50K}). Замените целевую переменную на числовые значения.

In [571]:
df = data.copy()
df[14] = df[14].map({'<=50K': 0, '>50K': 1})
x, y = df.drop(columns=14), df[14]

9. Выясните, присутствуют ли в данных пропуски. Заполните их самыми частыми значениями (испольуйте SimpleImputer)

In [518]:
df.isna().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
dtype: int64

10. Выберите колонки с числовыми и категориальными переменными.

In [535]:
numeric_columns = x.select_dtypes('int64').columns
cat_columns = x.select_dtypes('object').columns
# проверим, что разделили все калонки на категориальные и числовые
print(not bool(set(x.columns) - (set(numeric_columns) | set(cat_columns))))

True


11. Создайте пайплайн по обработке колонок(используйте OneHotEncoder,MinMaxScaler).

In [537]:
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
transformer = ColumnTransformer([
    ('categorical', OneHotEncoder(), cat_columns),
    ('numeric', MinMaxScaler(), number_columns)
])
pipe = Pipeline([('transformer', transformer)])
x_pipe = pipe.fit_transform(x)

12. Посчитайте метрики accuracy и f1_score на предсказании только самого частого класса в целевой переменной.

In [538]:
from sklearn.neighbors import KNeighborsClassifier
k_neigh = KNeighborsClassifier()
k_neigh.fit(x_pipe, y)
y_predict = k_neigh.predict(x_pipe)
y_predict

array([0, 0, 0, ..., 1, 1, 1])

In [539]:
from sklearn.metrics import accuracy_score, f1_score
# выберем в y test только самую популярную целевую переменую
most_common_y = y.value_counts().idxmax()
y_common = y[y == most_common_y].astype('int')
y_predict_common = y_predict[y_common.index.tolist()]
print('accuracy = ', accuracy_score(y_common, y_predict_common))
print('f1 score = ', f1_score(y_common, y_predict_common))

accuracy =  0.9379625891535459
f1 score =  0.0


13. Посчитайте cross_val_score по алгоритмам LogisticRegression, SVC, LinearSVC по метрикам accuracy и f1_score.
Напишите удалось ли превзойти предыдущий результат.

In [540]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC

In [541]:
print(cross_val_score(LogisticRegression(), x_pipe, y, cv=3, scoring='accuracy'))
print(cross_val_score(LogisticRegression(), x_pipe, y, cv=3, scoring='f1'))

[0.84847368 0.85394018 0.85067568]
[0.65051707 0.66355405 0.65473654]


In [542]:
print(cross_val_score(SVC(), x_pipe, y, cv=3, scoring='accuracy'))
print(cross_val_score(SVC(), x_pipe, y, cv=3, scoring='f1'))

[0.83545237 0.83926049 0.84287469]
[0.60839059 0.61463702 0.62689615]


In [543]:
print(cross_val_score(LinearSVC(), x_pipe, y, cv=3, scoring='accuracy'))
print(cross_val_score(LinearSVC(), x_pipe, y, cv=3, scoring='f1'))

[0.85086911 0.85486149 0.85288698]
[0.65462304 0.66276581 0.65712241]


14. Можно заметить что в данных присутствуют значения '?', замените их самыми частыми значениями (испольуйте SimpleImputer)

In [544]:
from sklearn.impute import SimpleImputer

si = SimpleImputer(fill_value='?', strategy='most_frequent')
x_si = si.fit_transform(x)
x_si_pipe = pipe.fit_transform(x_si)

15. Посчитайте cross_val_score на новых данных. Напишите удалось ли улучшить результат.

In [545]:
print(cross_val_score(LogisticRegression(), x_si_pipe, y, cv=3, scoring='accuracy'))
print(cross_val_score(LogisticRegression(), x_si_pipe, y, cv=3, scoring='f1'))

[0.84847368 0.85394018 0.85067568]
[0.65051707 0.66355405 0.65473654]


In [546]:
print(cross_val_score(SVC(), x_si_pipe, y, cv=3, scoring='accuracy'))
print(cross_val_score(SVC(), x_si_pipe, y, cv=3, scoring='f1'))

[0.83545237 0.83926049 0.84287469]
[0.60839059 0.61463702 0.62689615]


In [547]:
print(cross_val_score(LinearSVC(), x_si_pipe, y, cv=3, scoring='accuracy'))
print(cross_val_score(LinearSVC(), x_si_pipe, y, cv=3, scoring='f1'))

[0.85086911 0.85486149 0.85288698]
[0.65462304 0.66276581 0.65712241]


16. Посчитайте cross_val_score, если просто удалить значения '?'. Напишите как изменился результат

In [548]:
df_prepared = df.replace('?', pd.NA).dropna()
x_prep, y_prep = df_prepared.drop(columns=14), df_prepared[14]
x_prep_pipe = pipe.fit_transform(x_prep)

In [549]:
print(cross_val_score(LogisticRegression(), x_prep_pipe, y_prep, cv=3, scoring='accuracy'))
print(cross_val_score(LogisticRegression(), x_prep_pipe, y_prep, cv=3, scoring='f1'))

[0.84297466 0.85086905 0.84549556]
[0.65175813 0.66911981 0.65613465]


In [550]:
print(cross_val_score(SVC(), x_prep_pipe, y_prep, cv=3, scoring='accuracy'))
print(cross_val_score(SVC(), x_prep_pipe, y_prep, cv=3, scoring='f1'))

[0.83056919 0.83494759 0.83726947]
[0.6147813  0.62130898 0.63107234]


In [551]:
print(cross_val_score(LinearSVC(), x_prep_pipe, y_prep, cv=3, scoring='accuracy'))
print(cross_val_score(LinearSVC(), x_prep_pipe, y_prep, cv=3, scoring='f1'))

[0.84589359 0.85193048 0.84748574]
[0.65620838 0.66933333 0.65905383]


 17. Посчитайте cross_val_score для RandomForestClassifier,GradientBoostingClassifier. Напишите как изменился результат и какой вывод можно из этого сделать.

In [552]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
print(cross_val_score(RandomForestClassifier(), x_pipe, y, cv=3, scoring='accuracy'))
print(cross_val_score(GradientBoostingClassifier(), x_pipe, y, cv=3, scoring='accuracy'))

[0.85277317 0.85535287 0.84981572]
[0.86432037 0.86874271 0.86818182]


In [615]:
# По результатам видно, что GradientBoostingClassifier дает лучший результат.

18. Подберите наилучшую модель, подбирая методы обработки колонок - масштабирование признаков, кодирование признаков и заполнение пропусков. Параметры алгоритмов оставьте по умолчанию. Выведите итоговые параметры и результат accuracy и f1_score.

In [606]:
def make_pipeline(inputer, scaler, encoder, model, x, y):
    transformer = ColumnTransformer([
        ('categorical', encoder, cat_columns),
        ('numeric', scaler, number_columns)
    ])
    pipe = Pipeline([('transformer', transformer)])
    x_pipe = pipe.fit_transform(x)
    return cross_val_score(model, x_pipe, y, scoring='accuracy')

In [607]:
from sklearn.preprocessing import OrdinalEncoder
result = []
models = (RandomForestClassifier, GradientBoostingClassifier, LogisticRegression, SVC)
inputers = ('median', 'mean', 'most_frequent')
encoders = (OneHotEncoder, OrdinalEncoder)
scalers = (MinMaxScaler, StandardScaler)
for model in models:
    for inputer in inputers:
        for encoder in encoders:
            for scaler in scalers:
                acc = make_pipeline(
                    inputer=SimpleImputer(fill_value='?', strategy=inputer),
                    encoder=encoder(), 
                    scaler=scaler(),
                    model=model(),
                    x=x,
                    y=y
                )
                result.append((f'{model}/{inputer}/{encoder}/{scaler}', acc, f1))

In [613]:
result_ = [(r[0], max(r[1])) for r in result]
max(result_, key=lambda x: x[1])

("<class 'sklearn.ensemble._gb.GradientBoostingClassifier'>/mean/<class 'sklearn.preprocessing._encoders.OrdinalEncoder'>/<class 'sklearn.preprocessing._data.MinMaxScaler'>",
 0.8715192465192465)

In [614]:
# Таким образом, запустив все модели, наилучший результат показала модель GradientBoostingClassifier с заполнением пропуском - средним
# c OrdinalEncoder и масштабированием MinMax